# **The Datasets Library**
**Deep Dive:**
- What do you do when your **dataset is not on the Hub**?
- How can you **slice and dice** a dataset? (And what if you really need to use Pandas?)
- What do you do when your dataset is huge and will **melt your laptop’s RAM**?
- What the heck are **“memory mapping” and Apache Arrow**?
- How can you **create your own dataset** and push it to the Hub?

In [2]:
!python --version

Python 3.10.12


In [3]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
import torch
import transformers
from transformers import pipeline
from transformers import AutoModel, AutoTokenizer
from datasets import Dataset, load_dataset

import warnings
warnings.filterwarnings('ignore')

## **Not in the Hub**
`Datasets` can be used to load datasets that aren’t available on the Hugging Face Hub.

### **Working w/ Local and Remote Datasets**
Loading script by using `load_dataset()`:
`csv`, `text`, `json`, *Pickled DataFrames* `pandas`. Specify the type of script and `data_files` of specific files.

#### **Loading a Local Dataset**
For this example we’ll use the **SQuAD-it** dataset, which is a large-scale dataset for question answering in Italian.

The training and test splits are hosted on GitHub, so we can download them with a simple `wget` command

In [ ]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

This will download two compressed files called `SQuAD_it-train.json.gz` and `SQuAD_it-test.json.gz`, which we can **decompress** with the Linux `gzip` command

In [ ]:
!gzip -dkv SQuAD_it-*.json.gz

Load a `JSON` (nested dictionary) file. `SQuAD-it` uses the nested format, with all the text stored in a data field. This means we can load the dataset by specifying the field argument

In [ ]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

By default, loading local files creates a `DatasetDict` object with a train split

In [ ]:
squad_it_dataset

In [ ]:
squad_it_dataset["train"][0]

Include both the `train` and `test` split to a single `DatasetDict`, so we can apply `dataset.map()` functions accross both splits at once

In [ ]:
# maps each split name
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
# load dataset with it data_files
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

**This is exactly what we wanted. Now, we can apply various preprocessing techniques to clean up the data, tokenize the reviews, and so on.**

Skipping the decompression (`gzip`) with `data_files`

In [ ]:
data_files = {"train": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

#### **Loading a Remote Dataset**
Fortunately, loading remote files is just as simple as loading local ones! Instead of providing a path to local files, we point the `data_files` argument of `load_dataset()` to one or more **URLs where the remote files are stored**.

In [ ]:
from datasets import load_dataset

# Location where the dataset stored
url = "https://github.com/crux82/squad-it/raw/master/"

# Combine the file with the url (location of the dataset)
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}

# Load dataset as before
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [ ]:
squad_it_dataset['train'][0]

{'title': 'Terremoto del Sichuan del 2008',
 'paragraphs': [{'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
   'qas': [{'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56cdca7862d2951400fa6826',
     'question': 'In quale anno si è verificato il terremoto nel Sichuan?'},
    {'answers': [{'answer_start': 232, 'text': '69.197'}],
     'id': '56cdca7862d2951400fa6828',
     'question': 'Quante persone sono state uccise come risultato?'},
    {'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56d4f9902ccc5a1400d833c0',
     'question': 'Quale anno ha avuto luogo il terremoto del Sichuan?'},
    {'answers': [{'answer_start': 78, 'text': '8.0 Ms e 7.9 Mw'}],
     'id': '56d4f9902ccc5a1400d833c1',
     'question': 'Che cosa ha

## **Slice and Dice**
**Data Wrangling** ||| Most of the time, the data you work with won’t be perfectly prepared for training models. **Clean up the Datasets**

🎯 Manipulate the contents of `Dataset` and `DatasetDict` object.

### **Slicing and Dicing the Data**
For this example we’ll use the **Drug Review Dataset** that’s hosted on the **UC Irvine Machine Learning Repository**, which contains *patient reviews on various drugs, along with the condition being treated and a 10-star rating of the patient’s satisfaction*.

First we need to **download and extract the data**

In [2]:
!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2024-09-01 07:26:46--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [     <=>            ]  41.00M  41.1MB/s    in 1.0s    

2024-09-01 07:26:47 (41.1 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


Load these train and test dataset by using `csv` but with `delimeter`

In [3]:
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t") #\t tab

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [5]:
drug_dataset['train'][0]

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'Left Ventricular Dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27}

A **good practice when doing any sort of data analysis is to grab a small random sample** to get a quick feel for the type of data you’re working with.
- Create a **random sample** by chaining `.shuffle()` and `.select()` functions together

In [6]:
# reproducibility purporses || range(1000) -> grab first 1000 examples from the shuffled dataset
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))

# Peek at the first few examples
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

From this sample we can already see a few quirks in our dataset:

- The `Unnamed: 0` column looks suspiciously like an anonymized ID for each patient.
- The `condition` column includes a mix of uppercase and lowercase labels.
- The `reviews` are of varying length and contain a mix of Python line separators (`\r\n`) as well as HTML character codes like `&\#039;`.

**1. Test the patient ID hypothesis for the `Unnamed: 0` column** || Verify that the number of IDs matches the number of rows in each split

In [7]:
len(drug_dataset['train'].unique('Unnamed: 0'))

161297

In [8]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

## The hypothesis is True, if returns True
## otherwise it's False hten return AssertionError

-------- *rename the `Unnamed: 0` column*

In [9]:
drug_dataset = drug_dataset.rename_column('Unnamed: 0', 'patient_id')

drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

-------- *find the number of unique drug and conditions*

In [10]:
for split in drug_dataset.keys():
    for features in ['drugName', 'condition']:
        print(f"Unique {features} for {split} dataset: {len(drug_dataset[split].unique(features))}")

Unique drugName for train dataset: 3436
Unique condition for train dataset: 885
Unique drugName for test dataset: 2637
Unique condition for test dataset: 709


**2. Normalize all the `conditions` labels**

In [11]:
## Function to lowercase
def lowercase_condition(x):
    return {"condition": x["condition"].lower()}
    # return x['condition'] is not None

## Function to remove the None value
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [12]:
## Normalize the condition
drug_dataset = drug_dataset.map(lowercase_condition)

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [13]:
# Check that lowercasing worked
drug_dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

### **Creating New Columns**
`reviews` || Whenever you’re dealing with *customer reviews*, a good practice is to **check the number of words in each review**.

1. function to **counts the number of words** in each review

In [14]:
## Tons of words
drug_dataset["train"]["review"][:3]

['"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 '"I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects. But it contained hormo

In [27]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

In [28]:
drug_dataset = drug_dataset.map(compute_review_length)

# Inspect the first training example
drug_dataset["train"][3]

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'patient_id': 138000,
 'drugName': 'Ortho Evra',
 'condition': 'birth control',
 'review': '"This is my first time using any form of birth control. I&#039;m glad I went with the patch, I have been on it for 8 months. At first It decreased my libido but that subsided. The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control. Other than that in happy with the patch"',
 'rating': 8.0,
 'date': 'November 3, 2015',
 'usefulCount': 10,
 'review_length': 89}

Check the extreme values for the review length by using `.sort()`

In [29]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

some of the reviews contain just a single word, although it may be **okay for sentiment analysis**, *would not be informative if we want to predict the condition*.

2. Remove `reviews` that contain fewer than 30 words. (*filtering out aroung 15% of the reviews from the original*)

In [31]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

{'train': 138514, 'test': 46108}


In [ ]:
# drug_dataset['train'].sort('review_length', reverse=True)[:5]

3. **Remove HTML code**

In [32]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [33]:
# implement the unescape with .map() function
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

### **The map() method's superpowers**
The `Dataset.map()` method takes a `batched` argument that, if set to `True`, causes it to **send a batch of examples to the map function at once** (the batch size is configurable but defaults to 1,000)

#### **`batched=True`** || *faster `map()`*

In [34]:
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

 this command **executes way faster than the previous one**.

Using `Dataset.map()` with `batched=True` will be essential to unlock the speed of the **“fast”** tokenizers, which can **quickly tokenize big lists of texts**.

In [35]:
# Example -- tokenize all the drug reviews
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [36]:
# Check the performance of the function
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)
# %time tokenized_dataset = drug_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 1min 56s, sys: 1.42 s, total: 1min 58s
Wall time: 1min 26s


To check the different between **fast-tokenize**r and **slow-tokenzer** we can set the `use_fast=False` method.

In [38]:
# slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)

# try the Fast Tokenizer
fast_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Use **multprocessing** for `Dataset.map()` to activate the parallelization by use the `num_proc` argument.

In [39]:
def fast_tokenize_function(examples):
    return fast_tokenizer(examples["review"], truncation=True)

%time fast_tokenized_dataset = drug_dataset.map(fast_tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/138514 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 2.59 s, sys: 857 ms, total: 3.45 s
Wall time: 1min 47s


⚠ ⚠ ⚠ **In general, we don’t recommend using Python multiprocessing for fast tokenizers with `batched=True`**

----

📓 Using `num_proc` to speed up your processing is usually a great idea, as long as the function you are using is not already doing some kind of multiprocessing of its own.

#### **`overflowing`** || *changed the number of elements*
|| Create several training features from one example

💡 In machine learning, an *example* is usually defined as the set of features that we feed to the model.

In some contexts, these features will be the set of columns in a Dataset, but in others (like here and for question answering), multiple features can be extracted from a single *example* and belong to a single column.

In [40]:
# Function to tokenizer the examples
## Truncate them to a maximum length of 128
## return all the chunks of the texts
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

Test with one example

In [41]:
result = tokenize_and_split(drug_dataset["train"][0])
[len(inp) for inp in result["input_ids"]]

[128, 49]

So, our first example in the training set became two features because it was tokenized to more than the maximum number of tokens we specified:
- the first one of length 128 (*max*)
- the second one of length 49 (*overflow*).

In [42]:
drug_dataset['train'].column_names

['patient_id',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount',
 'review_length']

**Removing old dataset**

In [43]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [44]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [45]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 68876
    })
})

In [46]:
len(tokenized_dataset["train"]), len(drug_dataset["train"])

(206772, 138514)

**Making old columns the same size**, but still keep the old columns

In [47]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping") # mapping new feature index to the index of the original

    # list of values
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]

    return result

In [53]:
# Run without remove the old columns
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

We get the same number of training features as before, but here we’ve kept all the old fields.

If you need them for some post-processing after applying your model, you might want to use this approach.

### **From Datasets to DataFrame and back**
To enable the conversion between various third-party libraries, 🤗 Datasets provides a `Dataset.set_format(`) function.

📓 **changes the output format of the dataset, so you can easily switch to another format without affecting the underlying *data format***

In [54]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

**Convert dataset to Pandas**

In [55]:
drug_dataset.set_format("pandas")

In [56]:
drug_dataset["train"][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89


Create a `pandas.DataFrame` from the whole training set

In [57]:
train_df = drug_dataset["train"][:]

In [58]:
train_df.head()

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89
3,35696,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,124
4,155963,Cialis,benign prostatic hyperplasia,"""2nd day on 5mg started to work with rock hard...",2.0,"November 28, 2015",43,68


From here we can use all the Pandas functionality that we want.

For example, we can do fancy chaining to **compute the class distribution among the `condition` entries**.

In [59]:
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "condition": "frequency"})
)

frequencies.head()

,frequency,count
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


And once we’re done with our Pandas analysis, we can always create a new `Dataset` object by using the `Dataset.from_pandas()`

In [60]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['frequency', 'count'],
    num_rows: 819
})

Next, **compute the average rating per drug and store**.

In [62]:
train_df.columns

Index(['patient_id', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount', 'review_length'],
      dtype='object')

In [61]:
train_df.head(1)

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141


In [63]:
average_rating_per_drug = (
    train_df.groupby('drugName')['rating']
    .mean()
    .to_frame()
    .reset_index()
    .rename(columns={'drugName':'drug', 'rating':'average'})
)

average_rating_per_drug.head()

,drug,average
0,A + D Cracked Skin Relief,10.000000
1,A / B Otic,10.000000
2,Abacavir / dolutegravir / lamivudine,7.953488
3,Abacavir / lamivudine / zidovudine,9.000000
4,Abatacept,7.312500


In [64]:
avg_drug_rating = Dataset.from_pandas(average_rating_per_drug)
avg_drug_rating

Dataset({
    features: ['drug', 'average'],
    num_rows: 3052
})

In [65]:
drug_dataset.reset_format()

In [66]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Next, is to create a validation set to prepare the dataset for training a classifier on.

### **Creating a Validation set**
Although we have a test set we could use for evaluation, it’s a good practice to **leave the test set untouched** and create a **separate validation set during development**.

Once you are **happy with the performance of your models on the validation set**, you can **do a final sanity check on the test set**. This process helps mitigate the risk that you’ll overfit to the test set and deploy a model that fails on real-world data

In [67]:
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Let's use `Dataset.train_test_split()` to split the training data into train and validation

In [68]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)

# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")

# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]

drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

#### **Saving the Dataset**
- Arrow ➡ `Dataset.save_to_disk()`
- CSV ➡ `Dataset.to_csv()`
- JSON ➡ `Dataset.to_json()`

-------- **Save to Disk**

In [69]:
drug_dataset_clean.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

where we can see that each split is associated with its own `dataset.arrow` table, and some metadata in `dataset_info.json` and `state.json`.

You can think of the `Arrow` format as a fancy table of columns and rows that is optimized for building high-performance applications that process and transport large datasets.

**Load from the disk / local**

In [70]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("drug-reviews")
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

-------- **Save to JSON formats**

In [71]:
drug_dataset_clean.items()

dict_items([('train', Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
    num_rows: 110811
})), ('validation', Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
    num_rows: 27703
})), ('test', Dataset({
    features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
    num_rows: 46108
}))])

In [72]:
## split --> separate file (train, val, test)
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

This saves each split in `JSON Lines` format, where each row in the dataset is stored as a single line of `JSON`

In [73]:
!head -n 1 drug-reviews-train.jsonl

{"patient_id":89879,"drugName":"Cyclosporine","condition":"keratoconjunctivitis sicca","review":"\"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I've had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put the drops in it burns my eyes for the first 30 - 40 minutes.  I've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. Every year around spring time my eyes get terrible irritated  and this year has been the same (maybe even worse than other years) even though I've been using Restasis for a year now. The only difference I notice was for the first couple weeks, but now I'm ready to move on.\"","rating":2.0,"date":"April 20, 2013","usefulCount":69,"review_length":147}


Load the `JSON` files using `load_dataset()`

In [74]:
data_files = {
    "train": "drug-reviews-train.jsonl",
    "validation": "drug-reviews-validation.jsonl",
    "test": "drug-reviews-test.jsonl",
}

drug_dataset_reloaded = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [75]:
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

## **Big Data??**

### **What's the Pile**

### **The Magic of Memory Mapping**

### **Streaming Datasets**

## **Create Own Dataset**

## **Semantic Search with FAISS**